In [9]:
import os
os.chdir("/content/drive/MyDrive")

In [10]:
import numpy as np
specificity = list(np.load("Preprocessed Data/specificity.npy"))
clusters = list(np.load("TCR-RNA Data/clusters.npy"))
z_mean = list(np.load("TCR-RNA Data/z_mean.npy"))

In [11]:
from scipy.spatial.distance import pdist, squareform

# data是聚类后的数据
# labels是聚类后每个数据点所属的类别
data = z_mean
labels = clusters

distances = pdist(data)
dist_matrix = squareform(distances)

# 计算每个类别内部数据点的平均距离
cohesions = []
for label in set(labels):
    indices = [i for i, x in enumerate(labels) if x == label]
    dist_sum = sum([dist_matrix[i][j] for i in indices for j in indices])
    cohesion = dist_sum / (len(indices) * (len(indices) - 1) / 2)
    cohesions.append(cohesion)

In [12]:
print(set(labels))

{0, 1, 2, 3, 4}


In [13]:
mapping = ['A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 'A0301_KLGGALQAK_IE-1_CMV_binder', 'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder']
print(mapping)

['A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 'A0301_KLGGALQAK_IE-1_CMV_binder', 'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder']


In [14]:
print(cohesions)

[0.45219456021457277, 0.9606621992819183, 0.96225021763915, 0.9951394335642784, 0.9589359243132165]


In [15]:
data = np.array(data).reshape(-1, 2)
print(data)

[[-0.18428972  0.17507207]
 [-0.6145716  -1.5115254 ]
 [-1.2544881  -0.8997085 ]
 ...
 [ 0.17437893 -1.3105444 ]
 [ 0.7674321  -1.9533625 ]
 [-0.40364802 -0.6043943 ]]


In [16]:
# from scipy.spatial.distance import cdist

# # data是聚类后的数据
# # labels是聚类后每个数据点所属的类别

# separations = []
# for label1 in set(labels):
#     indices1 = [i for i, x in enumerate(labels) if x == label1]
#     for label2 in set(labels):
#         if label2 == label1:
#             continue
#         indices2 = [i for i, x in enumerate(labels) if x == label2]
#         dist_sum = sum([cdist(data[[i],:], data[[j],:]) for i in indices1 for j in indices2])
#         separation = dist_sum / (len(indices1) * len(indices2))
#         separations.append(separation)


In [17]:
# print(list(separations))

In [18]:
# sep = [1.54014793, 2.3612712, 3.29641642, 2.38029254, 1.54014793, 0.93127192, 1.93933423, 1.2643284, 2.3612712, 0.93127192, 1.26714071, 1.15798625, 3.29641642, 1.93933423, 1.26714071, 2.21278472, 2.38029254, 1.2643284, 1.15798625, 2.21278472]

In [19]:
# print(len(sep))
# sep

In [20]:
from sklearn import metrics
import copy
true_labels = specificity
labels_copy = copy.deepcopy(labels)
for i in range(len(labels_copy)):
  labels_copy[i] = mapping[labels_copy[i]]

In [21]:
f_measure_1 = metrics.f1_score(true_labels, labels_copy, average=None)
f_measure_2 = metrics.f1_score(true_labels, labels_copy, average='weighted')
print(f_measure_1)
print(f_measure_2)

[0.00310559 0.80091384 0.57935716 0.13194444 0.91057242]
0.6907506637498444


In [22]:
print(np.mean(cohesions))
# print(np.mean(sep))

0.8658364670026272


In [23]:
precision_list = []
recall_list = []
accuracy_list = []
iou_list = []
f1_list = []
for cluster in sorted(mapping):
  print(cluster)
  y_test = specificity
  y_pred = copy.deepcopy(clusters)
  for i in range(len(y_pred)):
    y_pred[i] = mapping[y_pred[i]]
  tn, fp, fn, tp = 0, 0, 0, 0
  for i in range(len(y_test)):
    if y_pred[i] == cluster and y_test[i] == cluster:
      tp += 1
    if y_pred[i] == cluster and y_test[i] != cluster:
      fp += 1
    if y_pred[i] != cluster and y_test[i] == cluster:
      fn += 1
    if y_pred[i] != cluster and y_test[i] != cluster:
      tn += 1

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  accuracy = (tp + tn) / ((tp + fn) + (fp + tn))
  iou = tp / (tp + fn + fp)
  f1 = (2 * tp) / (2 * tp + fn + fp)

  precision_list.append(precision)
  recall_list.append(recall)
  accuracy_list.append(accuracy)
  iou_list.append(iou)
  f1_list.append(f1)

  print("TP: {}".format(tp))
  print("TN: {}".format(tn))
  print("FP: {}".format(fp))
  print("FN: {}".format(fn))
  print()

  print("Precision: {:.4f}".format(precision))
  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("IoU: {:.4f}".format(iou))
  print("F1 Score: {:.4f}".format(f1))
  print()

print("Precision: {:.4f}".format(np.mean(precision_list)))
print("Recall: {:.4f}".format(np.mean(recall_list)))
print("Accuracy: {:.4f}".format(np.mean(accuracy_list)))
print("IoU: {:.4f}".format(np.mean(iou_list)))
print("F1 Score: {:.4f}".format(np.mean(f1_list)))

A0201_ELAGIGILTV_MART-1_Cancer_binder
TP: 1
TN: 7322
FP: 486
FN: 156

Precision: 0.0021
Recall: 0.0064
Accuracy: 0.9194
IoU: 0.0016
F1 Score: 0.0031

A0201_GILGFVFTL_Flu-MP_Influenza_binder
TP: 1227
TN: 6128
FP: 14
FN: 596

Precision: 0.9887
Recall: 0.6731
Accuracy: 0.9234
IoU: 0.6679
F1 Score: 0.8009

A0301_KLGGALQAK_IE-1_CMV_binder
TP: 1451
TN: 4407
FP: 944
FN: 1163

Precision: 0.6058
Recall: 0.5551
Accuracy: 0.7355
IoU: 0.4078
F1 Score: 0.5794

A1101_AVFDRKSDAK_EBNA-3B_EBV_binder
TP: 152
TN: 5813
FP: 1455
FN: 545

Precision: 0.0946
Recall: 0.2181
Accuracy: 0.7489
IoU: 0.0706
F1 Score: 0.1319

A1101_IVTDFSVIK_EBNA-3B_EBV_binder
TP: 2235
TN: 5291
FP: 0
FN: 439

Precision: 1.0000
Recall: 0.8358
Accuracy: 0.9449
IoU: 0.8358
F1 Score: 0.9106

Precision: 0.5382
Recall: 0.4577
Accuracy: 0.8544
IoU: 0.3968
F1 Score: 0.4852
